We first import the relevant packages

In [1]:
using LinearAlgebra: length
using ACE, ACEbase, Test, ACE.Testing
using ACE: evaluate, SymmetricBasis, PIBasis, O3, State, val 
using JuLIP
using StaticArrays
using ChainRules
import ChainRulesCore: rrule, NoTangent, ZeroTangent
using Zygote
using Zygote: @thunk 
using Printf, LinearAlgebra #for the fdtestMatrix

Now we construct the basis. Having maxdeg=6 and ord=3 gives us a symmetric basis of 97 elements. 

In [3]:
maxdeg = 6
ord = 3
Bsel = SimpleSparseBasis(ord, maxdeg)
B1p = ACE.Utils.RnYlm_1pbasis(; maxdeg=maxdeg)
φ = ACE.Invariant()
basis = SymmetricBasis(φ, B1p, O3(), Bsel)
length(basis)

97

Then we generate a random configuration, with the number of atoms being nX. cfg is then a vector of 97 elements, each element a DState (representing its position). 

In [4]:
nX = 10     # number of atoms
cfg = ACEConfig([State(rr = rand(SVector{3, Float64})) for _ in 1:nX])      # vector of States

ACEConfig{PositionState{Float64}}(PositionState{Float64}[⟨rr:[0.22, 0.44, 0.01]⟩, ⟨rr:[0.02, 0.4, 0.49]⟩, ⟨rr:[0.12, 0.73, 0.6]⟩, ⟨rr:[0.98, 0.36, 0.03]⟩, ⟨rr:[0.62, 0.5, 0.92]⟩, ⟨rr:[0.84, 0.07, 0.26]⟩, ⟨rr:[0.28, 0.88, 0.25]⟩, ⟨rr:[0.49, 0.91, 0.16]⟩, ⟨rr:[0.83, 0.52, 0.24]⟩, ⟨rr:[1.0, 0.48, 0.26]⟩])

We can now initialize the model (linear ACE model). The basis is stored in model.basis and the coefficients are stored in model.c. At this point what is the significance of 'np'? Since we have a basis of 97 polynomials, I would expect c_m to be a vector of 97 elements, but I find that it is $97 \times 2$ matrix. 

In [5]:
np = 2
c_m = rand(SVector{np,Float64}, length(basis))
model = ACE.LinearACEModel(basis, c_m, evaluator = :standard)

ACE.LinearACEModel{SymmetricBasis{PIBasis{ACE.Product1pBasis{2, Tuple{ACE.Rn1pBasis{Float64, PolyTransform{Int64, Float64}, ACE.OrthPolys.OrthPolyBasis{Float64}, :rr, :n, ACE.DState{NamedTuple{(:rr,), Tuple{SVector{3, Float64}}}}}, ACE.Ylm1pBasis{Float64, :rr, :l, :m, ACE.DState{NamedTuple{(:rr,), Tuple{SVector{3, ComplexF64}}}}}}, ComplexF64}, typeof(real), Float64, ComplexF64}, ACE.Invariant{Float64}, O3{:l, :m}, typeof(real), ACE.Invariant{Float64}}, SVector{2, Float64}, ACE.ProductEvaluator{SVector{2, ACE.Invariant{Float64}}, PIBasis{ACE.Product1pBasis{2, Tuple{ACE.Rn1pBasis{Float64, PolyTransform{Int64, Float64}, ACE.OrthPolys.OrthPolyBasis{Float64}, :rr, :n, ACE.DState{NamedTuple{(:rr,), Tuple{SVector{3, Float64}}}}}, ACE.Ylm1pBasis{Float64, :rr, :l, :m, ACE.DState{NamedTuple{(:rr,), Tuple{SVector{3, ComplexF64}}}}}}, ComplexF64}, typeof(real), Float64, ComplexF64}, typeof(real)}}(SymmetricBasis{PIBasis{ACE.Product1pBasis{2, Tuple{ACE.Rn1pBasis{Float64, PolyTransform{Int64, Float

Recall that our Finnis Sinclair model is: 
$$\hat{E} \big(\{\mathbf{r}_i \}, \mathbf{a}, \mathbf{a}^\prime \big) = \sum_i \sum_{k=1}^K  a_k \mathcal{B}_k \big( \{\mathbf{r}_{ij} \} \big) + \sum_i F\bigg( \sum_{k=1}^{K^\prime} a_k^\prime \mathcal{B}_k^\prime \big(\{\mathbf{r}_{ij} \} \big) \bigg) $$
I had suspected that the reason np=2 was that the first column of c_m represents the coefficients 97 $a_k$ and the second column represents the 97 coefficients $a_k^\prime$. But this wouldn't make sense since we are talking about a linear model. So I will need help explaining the significance of c_m being a $97 \times 2$ matrix. My best guess is that the $2$ columns of c_m represents two *independent* sets of coefficients (why that is necessary, I am not sure). 

Looking at the equation above, we should implement code that for the linear term, 
1. Takes the coefficients c_m and basis $\mathcal{B}_k (\{\mathbf{r}_{ij}\})$ 
2. Sums them all up over all the $nX$ atoms in at  

and for the nonlinear term, 
1. Takes the coefficients c_m and basis $\mathcal{B}_k (\{\mathbf{r}_{ij}\})$ 
2. Sums them all up over all the $nX$ atoms in at 